In [1]:
# 步骤1: 导入必要库
import os
import numpy as np
import faiss
import ollama
from tqdm import tqdm
import pickle
import time

In [4]:


# 步骤2: 配置常量
EMBEDDING_MODEL = "nomic-embed-text"
VECTOR_STORE_DIR = "./vector_stores"
INDEX_NAME = "my_documents"  # 替换为您的索引名称
MODEL_NAME = "llama3.2:latest" 

# 步骤3: 加载向量存储
def load_vector_store(index_name):
    index_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}.faiss")
    metadata_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}_metadata.pkl")
    
    try:
        index = faiss.read_index(index_path)
        with open(metadata_path, "rb") as f:
            data = pickle.load(f)
            chunks = data['chunks']
            metadata = data['metadata']
        print(f"✅ 成功加载向量存储 | 索引大小: {index.ntotal}")
        return index, chunks, metadata
    except Exception as e:
        print(f"❌ 加载失败: {str(e)}")
        return None, None, None

# 加载向量存储
index, chunks, metadata = load_vector_store(INDEX_NAME)

# 步骤4: 定义查询函数
def query_documents(question, index, chunks, metadata, model_name=MODEL_NAME, k=3):
    """查询文档并获取回答"""
    try:
        # 获取问题嵌入
        response = ollama.embeddings(model=EMBEDDING_MODEL, prompt=question)
        query_embedding = np.array([response['embedding']], dtype=np.float32)
        
        # 搜索相似内容
        distances, indices = index.search(query_embedding, k)
        
        # 获取相关文本块
        context_chunks = [chunks[i] for i in indices[0]]
        context_metadata = [metadata[i] for i in indices[0]]
        
        # 构造上下文
        context = "\n\n".join([
            f"来源: {meta['filename']} 第{meta['page']}页\n内容: {chunk}"
            for chunk, meta in zip(context_chunks, context_metadata)
        ])
        
        # 构造提示
        prompt = f"""
        基于以下上下文信息回答问题：
        {context}
        
        问题: {question}
        回答:
        """
        
        # 获取回答
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response['message']['content'], context_metadata
        
    except Exception as e:
        return f"查询失败: {str(e)}", []

# 步骤5: 交互式文档查询
print("🚀 文档助手已启动! 输入 'exit' 退出")

# while True:
#     try:
#         # 获取用户输入
#         question = input("\n👤 您的问题: ")
#         if question.lower() in ['exit', 'quit']:
#             print("再见!")
#             break
            
#         if not question.strip():
#             continue
            
#         # 查询文档
#         start_time = time.time()
#         answer, sources = query_documents(question, index, chunks, metadata)
#         elapsed = time.time() - start_time
        
#         # 显示结果
#         print(f"\n🤖 助手 (响应时间: {elapsed:.2f}s):")
#         print(answer)
        
#         # 显示来源
#         if sources:
#             print("\n📚 来源:")
#             for i, source in enumerate(sources):
#                 print(f"{i+1}. {source['filename']} - 第{source['page']}页")
    
#     except KeyboardInterrupt:
#         print("\n再见!")
#         break
#     except Exception as e:
#         print(f"❌ 错误: {str(e)}")

✅ 成功加载向量存储 | 索引大小: 60
🚀 文档助手已启动! 输入 'exit' 退出


In [6]:
# 直接在单元格中提问
question = "what is energy distance"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
根据提供的文本，Energy Distance是一个统计量，它被定义为：

E(X,Y) = E[Xd - Yd] - E[Xd - X'd] - E[Yd - Y'd]

其中，Xd、Yd、X'd 和 Y'd 分别代表 X、Y、X' 和 Y' 的 d 维 random variable。

在公式 (2.4) 中，E(X,Y) 表示 X 和 Y 的多元分布之间的距离。这个距离被称为 Energy Distance，它具有以下特性：

* 如果 X 和 Y 是相同分布，则 Energy Distance 为 0
* Energy Distance 是非负数
* Energy Distance 与其逆 (X'、Y') 分别独立和相互换位时的对称性有关

根据公式 (2.6)，Energy Distance 的表达式可以写成：

E(X,Y) = 1/2 cd \* [j^f(t) - j^g(t)]^2 dt

其中，cd 是一个常数，j^f(t) 和 j^g(t) 分别代表 X 和 Y 的特征函数。

📚 来源:
1. Energy statistics- A class of statistics based on distances.pdf - 第4页
2. Energy statistics- A class of statistics based on distances.pdf - 第3页
3. Energy statistics- A class of statistics based on distances.pdf - 第5页
